In [ ]:
import torch
from gensim.models import KeyedVectors
from torch import nn
from torch import optim
from torch.nn import functional
import utils

In [ ]:
# 设置超参数
batch_size = 128        # batch_size
seq_len = 100           #truncate_pad 后句子长度
lr = 1e-2               #学习率
num_epochs = 20         #总共训练轮数
specified_num_hiddens = 48  #隐藏层的单元个数
specified_num_layers = 3    #指定总共的层数
specified_drop_out = 0.4    #训练时，在线性层前面的drop_out层的drop_out力度

In [ ]:
# 预处理训练、测评数据
word_vectors = KeyedVectors.load_word2vec_format("./Dataset/wiki_word2vec_50.bin", binary=True)
#对于未知的词语，用随机一个vector表示
unk_vector = torch.randn(50, dtype=torch.float32) * 0.01
train_iter = utils.data_loader("./Dataset/train.txt", word_vectors, unk_vector, batch_size, True, seq_len)
test_iter = utils.data_loader("./Dataset/test.txt", word_vectors, unk_vector, batch_size, False, seq_len)
validation_iter = utils.data_loader("./Dataset/validation.txt", word_vectors, unk_vector, batch_size, False, seq_len)

In [ ]:
# 定义模型结构
class BiRNN_GRU(nn.Module):
    def __init__(self, num_hiddens, num_layers, drop_out=0, embed_size=50):
        super(BiRNN_GRU, self).__init__()
        self.encoder = nn.GRU(input_size=embed_size, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                bidirectional=True)
        self.dropout = nn.Dropout(drop_out)
        self.decoder = nn.Linear(4*num_hiddens, 2)

    def forward(self, inputs):
        outputs, _ = self.encoder(inputs.permute(1, 0, 2)) # output, (h, c)
        encoding = torch.cat((outputs[0], outputs[-1]), -1)
        return self.decoder(self.dropout(encoding))
    

In [ ]:
# 搭建模型，并进行训练、预测
net = BiRNN_GRU(num_hiddens=specified_num_hiddens, num_layers=specified_num_layers, drop_out=specified_drop_out)

net = utils.initialize_model(net, None)

model_path = "./saved_path/saved_GRU.pth"

optimizer = optim.Adam(net.parameters(), lr)
loss = nn.CrossEntropyLoss()
device = torch.device("cuda")
utils.train(train_iter, validation_iter, test_iter, net, loss, optimizer, device, num_epochs, model_path)

net.load_state_dict(torch.load(model_path))
test_acc, test_f_score = utils.eval_acc(net, test_iter, device)
print('total_epoch %d, test_acc %.3f, test_f_score, %.3f' % (num_epochs, test_acc, test_f_score))